In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from scipy import stats

from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold

from sklearn.preprocessing import StandardScaler

from scipy import stats

%matplotlib inline

I:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [332]:
train_data = pd.read_csv('house_price/train.csv')
train_data = train_data.drop(['Id'] , axis = 1)

In [333]:
#++++++++++++++++++
#++++++++++++++++++
test_data = pd.read_csv('house_price/test.csv')
test_id = test_data.Id
test_data = test_data.drop('Id' , axis = 1)

In [334]:
train_data.loc[train_data.GrLivArea>4500,'GrLivArea'].index #找离群点
train_data = train_data.drop([523 , 1298]) #上面已经把1298删除了
#注意删除前面的离群值后 后面的样本序号会变

In [335]:
#对saleprice进行除偏
train_data.SalePrice = train_data.SalePrice.apply(np.log1p)

In [336]:
train_label = train_data.SalePrice
train_data = train_data.drop('SalePrice' , axis =1)

In [337]:
new_data = pd.concat((train_data , test_data))
new_data = new_data.reset_index(drop=True) #drop参数的意思是将index排列正确
#new_data = new_data.drop('SalePrice' , axis = 1)

In [338]:
new_data.Alley = new_data.Alley.fillna('None')#
new_data.MasVnrType = new_data.MasVnrType.fillna('None')#

new_data.BsmtQual = new_data.BsmtQual.fillna('None')#
new_data.BsmtCond = new_data.BsmtCond.fillna('None')#
new_data.BsmtExposure = new_data.BsmtExposure.fillna('None')#
new_data.BsmtFinType1 = new_data.BsmtFinType1.fillna('None')#
new_data.BsmtFinType2 = new_data.BsmtFinType2.fillna('None')#

new_data.Electrical = new_data.Electrical.fillna(new_data.Electrical.value_counts().index[0])
new_data.FireplaceQu = new_data.FireplaceQu.fillna('None')#
new_data.GarageType = new_data.GarageType.fillna('None')#
new_data.GarageFinish = new_data.GarageFinish.fillna('None')#
new_data.GarageQual = new_data.GarageQual.fillna('None')#
new_data.GarageCond = new_data.GarageCond.fillna('None')#
new_data.PoolQC = new_data.PoolQC.fillna('None')#
new_data.Fence = new_data.Fence.fillna('None')#
new_data.MiscFeature = new_data.MiscFeature.fillna('None')#

new_data.LotFrontage = new_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))#
new_data.MasVnrArea = new_data.MasVnrArea.fillna(0)#
new_data.GarageYrBlt = new_data.GarageYrBlt.fillna(0)

new_data.GarageYrBlt = new_data.GarageYrBlt.fillna(0)#
new_data.GarageArea = new_data.GarageArea.fillna(0)#
new_data.GarageCars = new_data.GarageCars.fillna(0)#


for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    new_data[col] = new_data[col].fillna(0)

new_data.MSZoning = new_data.MSZoning.fillna(new_data.MSZoning.value_counts().index[0])


new_data = new_data.drop('Utilities' , axis=1)
new_data.Functional = new_data.Functional.fillna('Typ')

new_data.Electrical = new_data.Electrical.fillna(new_data.Electrical.value_counts().index[0])
new_data.KitchenQual = new_data.KitchenQual.fillna(new_data.KitchenQual.value_counts().index[0])

new_data.Exterior1st = new_data.Exterior1st.fillna(new_data.Exterior1st.value_counts().index[0])
new_data.Exterior2nd = new_data.Exterior2nd.fillna(new_data.Exterior2nd.value_counts().index[0])

new_data.SaleType = new_data.SaleType.fillna(new_data.SaleType.value_counts().index[0])

new_data.MSSubClass = new_data.MSSubClass.fillna('None')

#继续eda
#将数字变为str类型 为转换为categorical 类型做准备
new_data.MSSubClass = new_data.MSSubClass.apply(str)
new_data.OverallCond = new_data.OverallCond.astype(str)

new_data.YrSold = new_data.YrSold.astype(str)
new_data.MoSold = new_data.MoSold.astype(str)

In [339]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for i in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(new_data[i].values)) 
    new_data[i] = lbl.transform(list(new_data[i].values))



In [340]:
new_data['TotalSF'] = new_data['TotalBsmtSF'] + new_data['1stFlrSF'] + new_data['2ndFlrSF']

In [341]:
#=================
#boxcox除偏
numerical_features = new_data.dtypes[new_data.dtypes != object].index #含有预测值 SalePrice

skewed_feats = new_data[numerical_features].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
MiscVal,21.939672
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080
LandSlope,4.973254
KitchenAbvGr,4.300550
BsmtFinSF2,4.144503
EnclosedPorch,4.002344
ScreenPorch,3.945101


In [342]:
from scipy import special

for i in skewness[np.abs(skewness)>0.75].index:
    new_data[i] = special.boxcox1p(new_data[i] , 0.15)


In [343]:
new_data = pd.get_dummies(new_data)

In [344]:
new_data.shape

(2917, 220)

In [ ]:
#==============
#==============

#==============
#测试区域

In [345]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

forest = RandomForestRegressor(n_estimators=100)
linear = LinearRegression()
svr = SVR()
dec_tree = DecisionTreeRegressor()
knn = KNeighborsRegressor()
ridge = Ridge(alpha=3)
lasso = Lasso()

classifier = {'random_forest':forest , 'LR':linear , 'SVM':svr , 'decison_tree':dec_tree , 'KNN':knn , 'ridge':ridge , 'lasso':lasso}
#kf = KFold(train_data.shape[0] , 10 , True)

In [346]:
def rmse(model , data , label):
    #cv=5 分为5folds
    #在这里会对模型进行训练 
    return np.sqrt(-cross_val_score(model, data, label, scoring="neg_mean_squared_error", cv = 10))

In [347]:
best_model = _
smallest_rmse = 1.0
for model_name , model in classifier.items():
    result = np.average(rmse(model , new_data[:len(train_data)] , train_label))
    if result<smallest_rmse:
        smallest_rmse = result
        best_model = model_name
        
    print(model_name , result)

print('\nbest:',best_model , smallest_rmse)
#真大

random_forest 0.13528591269370702
LR 102789399.54489455
SVM 0.14804386972097486
decison_tree 0.20169016220159786
KNN 0.23192830567461917
ridge 0.11350319831128668
lasso 0.39884066356618436

best: ridge 0.11350319831128668


In [320]:
#================
#================
#投入使用
ridge.fit(new_data[:len(train_data)] , train_label)

Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [420]:
#进行预测
#生成结果
#result = ridge.predict(new_data[len(train_data):])
result = np.exp(result)
result_csv = pd.DataFrame({'Id':test_id , 'SalePrice':result})
result_csv.to_csv('submit2.csv' , index=False)

In [349]:
#kernel
from sklearn.linear_model import ElasticNet , Lasso  , BayesianRidge , LassoLarsIC
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator , TransformerMixin , RegressorMixin , clone
from sklearn.model_selection import KFold , cross_val_score , train_test_split
from sklearn.metrics import mean_squared_error


In [353]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds , shuffle=True , random_state=42).get_n_splits(train_data.values)
    rmse = np.sqrt(-cross_val_score(model  , new_data[:len(train_data)], train_label,
                                    scoring = 'neg_mean_squared_error' , cv=kf))
    
    return rmse

In [351]:
lasso = make_pipeline(RobustScaler()  , Lasso(alpha=0.0005 ,random_state=1))
ENet = make_pipeline(RobustScaler() , ElasticNet(alpha=0.0005 , l1_ratio=0.9,random_state=3))
KRR = KernelRidge(alpha=0.6 , kernel='polynomial' , degree=2 , coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000,learning_rate=0.05,
                                  max_depth = 4,max_features='sqrt',
                                  min_samples_leaf=15,min_samples_split=10,
                                  loss='huber',random_state=5)

In [356]:
rmsle_cv(lasso).mean()

0.11154335005030051

In [358]:
rmsle_cv(ENet).mean()

0.11156147457898787

In [359]:
rmsle_cv(KRR).mean()

0.11525466421169481

In [357]:
rmsle_cv(GBoost).mean()

0.11672026493514656

In [365]:
class averingmodels(BaseEstimator , RegressorMixin , TransformerMixin):
    def __init__(self , models):
        self.models = models
    
    def fit(self , X , y):
        self.models_ = [clone(x) for x in self.models]
        
        for model in self.models_:
            model.fit(X , y)
        
        return self
    
    def predict(self , X):
        predictions = np.column_stack([model.predict(X) for model in self.models_])
        
        return np.mean(predictions , axis = 1)
    



In [367]:
stacking_average_mode = averingmodels(models=(lasso , ENet , KRR , GBoost))
rmsle_cv(stacking_average_mode).mean()

0.10866239908682959

In [368]:
stacking_average_mode.fit(new_data[:len(train_data)] , train_label)

averingmodels(models=(Pipeline(steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   select...tors=3000, presort='auto', random_state=5,
             subsample=1.0, verbose=0, warm_start=False)))

In [369]:
result = stacking_average_mode.predict(new_data[len(train_data):])

In [414]:
class stackingaveragemodels(BaseEstimator , RegressorMixin , TransformerMixin):
    def __init__(self , base_models , meta_model , n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
    
    #对base model和meta model都需要进行训练
    def fit(self , X , y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds , shuffle=True , random_state=156)
        
        #预测值只是一个实数标量
        out_of_fold_predictions = np.zeros((X.shape[0] , len(self.base_models)))
        for i , model in enumerate(self.base_models):
            for train_index , holdout_index in kfold.split(X,y):
                instance = clone(model) #clone不影响原对象的任何属性
                self.base_models_[i].append(instance)
                instance.fit(X.iloc[train_index] , y.iloc[train_index])
                y_pred = instance.predict(X.iloc[holdout_index])
                out_of_fold_predictions[holdout_index , i] = y_pred
            
        self.meta_model_.fit(out_of_fold_predictions , y)
        return self
    
    def predict(self , X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1) 
            for base_models in self.base_models_])
        return self.meta_model_.predict(meta_features)
    
    


In [415]:
stacking_average_mode_2 = stackingaveragemodels(base_models=(ENet , GBoost , KRR) , meta_model=lasso)

In [416]:
rmsle_cv(stacking_average_mode_2).mean() #mean=0.10814471

array([0.1011876 , 0.11281233, 0.11986302, 0.10087643, 0.10598417])

In [410]:
new_data[:len(train_data)].iloc[[0,2]]

,MSSubClass,LotFrontage,LotArea,Street,Alley,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,2.885846,5.831328,19.212182,0.730463,0.730463,1.540963,0.0,2.440268,1.820334,14.187527,...,0,0,0,1,0,0,0,0,1,0
2,2.885846,5.914940,20.347241,0.730463,0.730463,0.000000,0.0,2.440268,1.820334,14.184404,...,0,0,0,1,0,0,0,0,1,0


In [417]:
stacking_average_mode_2.fit(new_data[:len(train_data)] , train_label)

stackingaveragemodels(base_models=(Pipeline(steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('elasticnet', ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=...nelRidge(alpha=0.6, coef0=2.5, degree=2, gamma=None, kernel='polynomial',
      kernel_params=None)),
           meta_model=Pipeline(steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))]),
           n_folds=5)

In [419]:
result = stacking_average_mode_2.predict(new_data[len(train_data):])